In [0]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import gc
import random

In [0]:
import h5py
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.losses import mean_squared_error as MSE
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/")
os.chdir("./data")


In [0]:
#helper code to check RAM and GPU resources

#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
#!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
path_to_imgs = "/content/drive/My Drive/comma_challenge/data/flow_imgs_train_cars/"
path_to_test_imgs = "/content/drive/My Drive/comma_challenge/data/flow_imgs_test_cars/"
path_to_trainY = "/content/drive/My Drive/comma_challenge/data/train.txt"
file_paths = sorted([os.path.join(path_to_imgs,f) for f in os.listdir(path_to_imgs)])
file_paths_test = sorted([os.path.join(path_to_test_imgs,f) for f in os.listdir(path_to_test_imgs)])

trainYs = []

with open(path_to_trainY, "r") as train_file:
  trainYs = train_file.read().splitlines()

trainYs = [float(line) for line in trainYs]

trainYs = np.asarray(trainYs[1:-1])
(train_files, test_files, trainY, testY) = train_test_split(file_paths,
	trainYs, test_size=0.20, random_state=42)

print(len(train_files))
print(len(test_files))


16318
4080


In [0]:
#adding frames from test images where the car is stopped (540 frames)
train_files.extend(file_paths_test[1098:1638])
trainY = np.concatenate((trainY, np.zeros([540])))
print(len(train_files))
print(len(trainY))


16858
16858


In [0]:
trainX = np.zeros([len(train_files), 74,220, 3])
testX = np.zeros([len(test_files), 74,220,3])


In [0]:
def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

In [0]:
!ls

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
checkpoints_withtest	      model_with_dropout_withtest_cars.json
checkpoints_withtest_cars     model_with_dropout_withtest.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test_bothflow_best.txt
flow_imgs_test_cars	      test.mp4
flow_imgs_train_cars	      test.txt
flow_test_imgs		      test_vert_best.txt
flow_test_imgs.tar	      test_vert.txt
horz_flow_imgs		      test_withtest_best.txt
horz_flow_imgs.tar	      train.mp4
horz_flow_test_imgs	      train.txt
horz_flow_test_imgs.tar       vert_flow_imgs
logs			      vert_flow_imgs.tar
model_with_dropout_both.h5    vert_flow_test_imgs
model_with_dropout_both.json  vert_flow_test_imgs.tar
model_with_dropout.h5


In [0]:
i = 0
for file in train_files:
  trainX[i] = cv2.imread(file, cv2.IMREAD_COLOR)
  if(i % 100 == 0):
    print(i)
  i += 1
print(i)
i = 0
for file in test_files:
  testX[i] = cv2.imread(file, cv2.IMREAD_COLOR)
  if(i % 100 == 0):
    print(i)
  i += 1
print(i)

In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/")
os.chdir("./data")
!mkdir checkpoints_withtest_cars

In [0]:
# helper code to save large files in Google drive
# Which file to send?
file_name = "LARGE_FILE"

from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {'name': name, 'mimeType': 'application/octet-stream'}
  media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)
  created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
  
  return created

save_file_to_drive(file_name, file_name)

In [0]:
trainX /= 255.0
testX /= 255.0


In [0]:
# output was gotten (i.e, this cell was run) after model was trained and checkpoints were created
!ls ./checkpoints_withtest_cars/

weights-improvement-01-6.98.hdf5  weights-improvement-09-0.91.hdf5
weights-improvement-02-2.97.hdf5  weights-improvement-12-0.85.hdf5
weights-improvement-03-2.32.hdf5  weights-improvement-14-0.65.hdf5
weights-improvement-04-1.95.hdf5  weights-improvement-16-0.59.hdf5
weights-improvement-06-1.43.hdf5  weights-improvement-17-0.52.hdf5
weights-improvement-07-1.15.hdf5  weights-improvement-21-0.36.hdf5


In [0]:
#Model hyperparameters

NUM_EPOCHS=25
opt='adam'
los='mse'
minibatch_size=32

In [0]:
#model architecture

model = Sequential()
model.add(Conv2D(24, (5,5), activation='relu', strides=2, data_format='channels_last', input_shape=(74,220,3)))
model.add(Conv2D(36, (5,5), activation='relu', strides=2, data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Conv2D(48, (5,5), activation='relu', strides=2, data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3), activation='relu', data_format='channels_last'))
model.add(Conv2D(64, (3,3), activation='relu', data_format='channels_last'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))


In [0]:
checkpoint_path = "/content/drive/My Drive/comma_challenge/data/checkpoints_withtest_cars/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
model_path = "/content/drive/My Drive/comma_challenge/data/model_with_dropout_withtest_cars.json"
log_path="/content/drive/My Drive/comma_challenge/logs"
os.chdir("/content/drive/My Drive/comma_challenge/data/")

In [0]:
!ls

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
checkpoints_withtest	      model_with_dropout_withtest.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test_bothflow_best.txt
flow_test_imgs		      test.mp4
flow_test_imgs.tar	      test.txt
horz_flow_imgs		      test_vert_best.txt
horz_flow_imgs.tar	      test_vert.txt
horz_flow_test_imgs	      train.mp4
horz_flow_test_imgs.tar       train.txt
logs			      vert_flow_imgs
model_with_dropout_both.h5    vert_flow_imgs.tar
model_with_dropout_both.json  vert_flow_test_imgs
model_with_dropout.h5	      vert_flow_test_imgs.tar


In [0]:
adam = Adam(lr=1e-4, beta_1=1, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer="adam", loss=los, metrics=['mse', 'acc'])
checkpt = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
tboard = TensorBoard(log_dir=log_path, write_graph=False)
callbacks_list = [checkpt]


In [0]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=NUM_EPOCHS, batch_size=minibatch_size, callbacks=callbacks_list)

Train on 16858 samples, validate on 4080 samples
Epoch 1/25
16858/16858 [==============================] - 20s 1ms/step - loss: 20.9140 - mean_squared_error: 20.9140 - acc: 0.0098 - val_loss: 6.9837 - val_mean_squared_error: 6.9837 - val_acc: 0.0333

Epoch 00001: val_loss improved from inf to 6.98372, saving model to /content/drive/My Drive/comma_challenge/data/checkpoints_withtest_cars/weights-improvement-01-6.98.hdf5
Epoch 2/25
16858/16858 [==============================] - 15s 902us/step - loss: 4.5781 - mean_squared_error: 4.5781 - acc: 0.0422 - val_loss: 2.9682 - val_mean_squared_error: 2.9682 - val_acc: 0.0348

Epoch 00002: val_loss improved from 6.98372 to 2.96815, saving model to /content/drive/My Drive/comma_challenge/data/checkpoints_withtest_cars/weights-improvement-02-2.97.hdf5
Epoch 3/25
16858/16858 [==============================] - 15s 887us/step - loss: 2.8951 - mean_squared_error: 2.8951 - acc: 0.0503 - val_loss: 2.3235 - val_mean_squared_error: 2.3235 - val_acc: 0.031

In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/data/")
model_json = model.to_json()
with open(model_path, "w") as file:
  file.write(model_json)

In [0]:
!ls "./flow_imgs_test_cars" | wc -l

10796


In [0]:
file_paths_test = sorted([os.path.join(path_to_test_imgs,f) for f in os.listdir(path_to_test_imgs)])
test_paths = file_paths_test


In [0]:
from keras.models import model_from_json

os.chdir("/content/drive/My Drive/comma_challenge/data")
json_file = open('model_with_dropout_withtest_cars.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./checkpoints_withtest_cars/weights-improvement-21-0.36.hdf5")
print("Loaded model from disk")

loaded_model.compile(loss='mse', optimizer='adam')

Loaded model from disk


In [0]:
#final test file. Changed the name to test.txt manually later
with open("test_withtestcars_best.txt", 'w') as f:
  for file in test_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR) / 255.0
    img = np.reshape(img, [1, img.shape[0], img.shape[1], img.shape[2]])
    pred = max(loaded_model.predict(img)[0,0], 0.0) 
    pred_str = str(pred)
    f.write(pred_str + '\n')